In [70]:
#Bayesian classifier
import skimage.transform
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import math

from skimage import data, color
from skimage.transform import rescale, resize

def unpickle(file):
    with open(file, 'rb') as f:
        dict = pickle.load(f, encoding="latin1")
    return dict

#datadict = unpickle('/home/kamarain/Data/cifar-10-batches-py/data_batch_1')
datadict = unpickle('C:/Users/danie/Desktop/TUNI/ML-PatternRec/EX2/cifar-10-batches-py/test_batch')

X = datadict["data"]
Y = datadict["labels"]

#labeldict = unpickle('/home/kamarain/Data/cifar-10-batches-py/batches.meta')
labeldict = unpickle('C:/Users/danie/Desktop/TUNI/ML-PatternRec/EX2/cifar-10-batches-py/batches.meta')
label_names = labeldict["label_names"]


#X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("uint8")
X = X.reshape(len(X), 3, 32, 32).transpose(0, 2, 3, 1).astype("uint8")
##np.resize


input_directory= ('C:/Users/danie/Desktop/TUNI/ML-PatternRec/EX2/cifar-10-batches-py/')
train_1= unpickle(input_directory+'data_batch_1')
train_2= unpickle(input_directory+'data_batch_2')
train_3= unpickle(input_directory+'data_batch_3')
train_4= unpickle(input_directory+'data_batch_4')
train_5= unpickle(input_directory+'data_batch_5')

#train_X=np.append(train_1["data"],train_2["data"],train_3["data"],train_4["data"],train_5["data"],axis=0)
train_X=np.append(train_1["data"],train_2["data"],axis=0)
train_X=np.append(train_X, train_3["data"],axis=0)
train_X=np.append(train_X, train_4["data"],axis=0)
train_X=np.append(train_X, train_5["data"],axis=0)


#print(type(train_1["labels"]))
train_Y=train_1["labels"]
train_Y.extend(train_2["labels"])
train_Y.extend(train_3["labels"])
train_Y.extend(train_4["labels"])
train_Y.extend(train_5["labels"])

train_Y = np.array(train_Y)
Y = np.array(Y)
print(len(train_Y))


50000


In [71]:
def cifar_10_color(X,n=1):
    color_out=[]
    X_=X.copy()
    X_ = X_.reshape(X_.shape[0], 3, 32, 32).transpose(0,2,3,1).astype(np.uint8)
    for i in range(X_.shape[0]):
        q=transform.resize(X_[i],(n,n)).reshape(n*n*3)
        color_out.append(q)
    return np.array(color_out)
    

In [72]:
def cifar_10_naivebayes_learn(X,Y,isNaive=True):
    mean=[]
    variance=[]
    priori=[]
    covariance=[]
    data=np.c_[X,Y]
    for i in range(len(label_names)):
        filter =np.array([i])
        mask = np.in1d(data[:, data.shape[1]-1], filter)
        data1=data[mask]
        data_t=[]
        cur_mean=[]
        for j in range(data1.shape[1]-1):
            cur_mean.append(data1[:,j].mean())
           
            if not isNaive:
                data_t.append(data1[:,j])
        mean.append(cur_mean)
        priori.append(data1.shape[0]/Y.shape[0])
        cur_var=[]
        data_t=np.array(data_t)
      
        for j in range(data1.shape[1]-1):
            if isNaive:
                cur_var.append(data1[:,j].var())
        if isNaive:
            variance.append(cur_var)
        else:
            variance.append(np.cov(data_t,bias=True))

    return np.array(mean),np.array(variance),np.array(priori)

IndexError: too many indices for array

In [ ]:
def gaussian_pdf(x,mean,var):
      return (1/math.sqrt(2*math.pi*var))*math.exp(-0.5*((x-mean)**2/var))

In [ ]:
def multivariate_normal_DIY(x, d, mean, covariance):
    x_m = x - mean
    return (1. / (mp.sqrt((2 * np.pi)**d * np.linalg.det(covariance))) * mp.exp(-(np.linalg.solve(covariance, x_m).T.dot(x_m)) / 2))

In [ ]:
def cifar10_classifier_naivebayes(X,mean,var,prioric,isNaive=True):
    Y=[]
    for i in range(len(X)):
        p_rgb=[]
        for j in range(len(label_names)):
            if isNaive:
                temp_ijk_p=1
                for k in range(mean.shape[1]):
                    ijk_p=gaussian_pdf(X[i,k],mean[j,k],var[j,k])
                    temp_ijk_p=temp_ijk_p*ijk_p
                p_rgb.append(temp_ijk_p*prioric[j])
            else:
                #current_var=multivariate_normal_DIY(X[i], var[j].shape[0], mean[j], var[j])
                current_var = multivariate_normal(mean=mean[j], cov=var[j]).logpdf(X[i])
                #print("@@2label===>"+str(j)+" current_var is ==> "+str(current_var))
                p_rgb.append(current_var*prioric[j])
            
        sum_rgb=sum(p_rgb)
        max_p=-1
        max_label=None
        for j in range(len(label_names)):
            current_p=p_rgb[j]/sum_rgb
            #print("label===>"+str(j)+" current_p is ==> "+str(current_p)+"p_rgp==> "+str(p_rgb))
            if current_p>max_p:
                max_p=current_p
                max_label=j
        #print("label for ==> "+str(i) +"is ==> "+str(max_label)+" its max prob ==> "+str(max_p))
        Y.append(max_label)
        
    return np.array(Y)

In [ ]:
def class_acc(pred,gt):
    temp=pred-gt
    temp1=temp[temp==0]
    return (temp1.size/pred.size)*100

In [ ]:
train_rgb_mean=cifar_10_color(train_X,n=1)
mean,var,prioric=cifar_10_naivebayes_learn(train_rgb_mean,train_Y)
test_rgb_mean=cifar_10_color(test_X,n=1)
pred=cifar10_classifier_naivebayes(test_rgb_mean,mean,var,prioric)
acc=class_acc(pred,test_Y)
print(" naive bayes algo - accuracy in percentage ==> "+str(acc))

In [ ]:
train_rgb_mean=cifar_10_color(train_X)
mean,var,prioric=cifar_10_naivebayes_learn(train_rgb_mean,train_Y,isNaive=False)
test_rgb_mean=cifar_10_color(test_X)
pred=cifar10_classifier_naivebayes(test_rgb_mean,mean,var,prioric,isNaive=False)
acc=class_acc(pred,test_Y)
print("  bayes algo (multivariate) - accuracy in percentage ==> "+str(acc))

In [ ]:
N=list(range(2,16))
new_N=[1]
acc_list=[acc]
for i in N:
    try:
        train_rgb_mean=cifar_10_color(train_X,n=i)
        mean,var,prioric=cifar_10_naivebayes_learn(train_rgb_mean,train_Y,isNaive=False)
        test_rgb_mean=cifar_10_color(test_X,n=i)
        pred=cifar10_classifier_naivebayes(test_rgb_mean,mean,var,prioric,isNaive=False)
        acc=class_acc(pred,test_Y)
        print("  bayes algo (multivariate) with n==> "+str(i)+" accuracy in percentage ==> "+str(acc))
        acc_list.append(acc)
        new_N.append(i)
    except:
        print("exception occurer while N is ==> "+str(i)+" skipping for rest of the N values")
        break

In [ ]:
plt.plot(np.array(new_N),np.array(acc_list))

NameError: name 'colorsCompressed' is not defined

NameError: name 'mu' is not defined

SyntaxError: invalid syntax (<ipython-input-79-c5ce340c9f71>, line 1)